In [ ]:
# В блокноте производится предобработка исходных данных асмпп по одному мвн.

# Предобработка заключается в следующем:
# 1. Выделение в исходных данных отдельных рейсов.
# 2. Выделение всех возможных вариантов поостановочной трассы.
# 3. Разделение по временным периодам. Выделены следующие категории временных периодов:
#    - раб/вых -- разделение в соответсвии со справочником рабочих дней;
#    - лето/зима -- лето: с 1 мая по 30 сентября, зима с 1 октября по 30 апреля (можно менять);
#    - час суток ("ЧЧ:ММ-ЧЧ:ММ") -- привязка рейса к часу суток осуществляется по времени прибытия на первый о.п. трассы
#    Временной период задается строкой вида <сезон>-<тип дня>-<час суток>, например, "лето-раб-08:00-08:59"
# 3. Вычисление числа рейсов по каждому варианту поостановочной трассы в указанный временной период.
# 4. Вычисление посадки-высадки среднего рейса.
# 
# В результате создается набор таблиц, содержащих следующие сведения:
# 1. Таблица inout -- посадка-высадка:
#    - mvn -- название мвн;
#    - trace_id -- идентификационный номер варианта поостановочной трассы (см. табл. traces);
#    - period_type -- тип временного периодов;
#    - stop_sequence -- порядковый номер о.п.;
#    - stop_id -- идентификационный номер о.п.;
#    - stop_name -- наименование о.п.;
#    - pass_in -- посадка;
#    - pass_out -- высадка.
# 2. Таблица traces -- варианты поостановочных трасс:
#    - trace_id -- идентификационный номер варианта поостановочной трассы;
#    - n_races -- общее число рейсов в исходных данных асмпп;
#    - date_first -- дата начала первого рейса в исходных данных асмпп;
#    - date_last -- дата начала последнего рейса в исходных данных асмпп;
#    - flag -- флаг использования данных о посаждке-высадке для построения маршрутной матрицы (1 -- используется);
#    - trip_id -- перечень через запятую trip_id имеющих указанную поостановочную трассу;
#    - trace -- поостановочная трасса составленная из последовательного перечня "<stop_sequence>-<stop_id>", разделенного ",\n".
# 3. Таблица nraces -- число рейсов по типам временных интервалов:
#    - первые три поля -- категории типов временных периодов:
#      - season -- лето/зима
#      - workday -- раб/вых
#      - time -- час суток
#    - остальные поля таблицы -- варианты поостановочной трассы мвн, в названии поля указан trace_id (см. табл. traces);
#    - значения -- число рейсов в исходных данных асмпп.
# 4. Таблица pass -- число перевезенных пассажиров в среднем рейсе по типам временных интервалов:
#    - первые три поля -- категории типов временных периодов:
#      - season -- лето/зима
#      - workday -- раб/вых
#      - time -- час суток
#    - остальные поля таблицы -- варианты поостановочной трассы мвн, в названии поля указан trace_id (см. табл. traces);
#    - значения -- число пассажиров в среднем рейсе асмпп в формате "<среднее число пассажиров> ± <ср. кв. откл.>".
#


In [1]:
import os
import sys
sys._enablelegacywindowsfsencoding()
import numpy as np
import pandas as pd

from tqdm import tqdm, trange
from tqdm import tqdm_notebook as tqdm

# %load_ext iimport
# %matplotlib inline

import geopandas as gpd
from sqlalchemy import create_engine
from fiona.crs import from_epsg

from datetime import date, datetime, time

Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.


In [14]:
all_data_file = pd.read_excel('File_to_matrix.xlsx')
all_data_file

,время,дата обкатки,тип дня,гос.номер / номер борта,ФИО обкатчика,наименование маршрута,Направление,Круговые маршруты (примечание),№ остановки,Наименование остановки,...,Геогр. широта,Геогр. долгота,вошло,вышло,в салоне,ID рейса,Время рейса,время начала рейса,время окончания рейса,Период
0,09:48:07,2018-11-14,буд,р000ах,Чудинова О,162,Обратно,NaN,1,пос. Обидимо,...,54.306066,37.424022,2,0,2,1,00:41:13.746000,09:48:07,10:29:20.746000,Межпик
1,09:48:39,2018-11-14,буд,р000ах,Чудинова О,162,Обратно,NaN,2,Парковая,...,54.297935,37.441982,0,0,2,1,00:41:13.746000,09:48:07,10:29:20.746000,Межпик
2,09:49:11,2018-11-14,буд,р000ах,Чудинова О,162,Обратно,NaN,3,Больница,...,54.293729,37.445221,1,0,3,1,00:41:13.746000,09:48:07,10:29:20.746000,Межпик
3,09:49:43,2018-11-14,буд,р000ах,Чудинова О,162,Обратно,NaN,4,Микрорайон,...,54.291530,37.450947,1,0,4,1,00:41:13.746000,09:48:07,10:29:20.746000,Межпик
4,09:50:52,2018-11-14,буд,р000ах,Чудинова О,162,Обратно,NaN,5,Совхозная,...,54.289768,37.455553,1,0,5,1,00:41:13.746000,09:48:07,10:29:20.746000,Межпик
5,09:54:28,2018-11-14,буд,р000ах,Чудинова О,162,Обратно,NaN,6,Рынок,...,54.286594,37.464210,2,2,5,1,00:41:13.746000,09:48:07,10:29:20.746000,Межпик
6,09:55:45.417000,2018-11-14,буд,р000ах,Чудинова О,162,Обратно,NaN,7,Каньшино,...,54.276950,37.471729,0,0,5,1,00:41:13.746000,09:48:07,10:29:20.746000,Межпик
7,09:56:31.614000,2018-11-14,буд,р000ах,Чудинова О,162,Обратно,NaN,8,Гараж,...,54.273641,37.476968,0,0,5,1,00:41:13.746000,09:48:07,10:29:20.746000,Межпик
8,09:57:38.621000,2018-11-14,буд,р000ах,Чудинова О,162,Обратно,NaN,9,Шоссейная,...,54.267693,37.481476,0,0,5,1,00:41:13.746000,09:48:07,10:29:20.746000,Межпик
9,10:01:28,2018-11-14,буд,р000ах,Чудинова О,162,Обратно,NaN,10,пос. Барсуки,...,54.262773,37.484562,1,0,6,1,00:41:13.746000,09:48:07,10:29:20.746000,Межпик


In [15]:
all_data_file['id маршрута'] = all_data_file['наименование маршрута']
all_data_file['id остановки'] = all_data_file['Наименование остановки']
all_data=all_data_file.rename(columns = {'id маршрута':'route_id',  'Stop_id':'stop_id',
                                    'Наименование остановки':'stop_name','№ остановки':'stop_sequence',
                                    'дата обкатки':'fact_trip_date','Направление':'direction','вошло':'pass_in',
                                    'вышло':'pass_out','ID рейса':'trip_num','наименование маршрута':'route_name',
                                    'время':'arrival_time','Период':'top_or_middle',
                                        'тип дня':'type_of_day'})

all_data['survey_id']=all_data['route_id']
all_data['trip_id']=all_data['route_id']
all_data['vehicle_id']=all_data['route_id']
all_data['grafic']=all_data['type_of_day']

all_data['survey_start_date']=all_data['fact_trip_date']
all_data['survey_end_date']=all_data['fact_trip_date']

all_data = all_data.replace({'direction': {'Вперед': 'Прямое', 'Обратно': 'Обратное'}})


all_data['str_mvn']=all_data[['route_name']].astype(str)

all_data["str_mvn"]=all_data.str_mvn.str.replace('Трм', 'Тм')
all_data["str_mvn"]=all_data.str_mvn.str.replace('Трл', 'Тб')


# new data frame with split value columns 
# new = all_data["str_mvn"].str.split("(", n = 1, expand = True) 
  
# making separate first name column from new data frame 
# all_data['str_mvn']= new[0]



def str_join(df, sep, *cols):
    from functools import reduce
    return reduce(lambda x, y: x.astype(str).str.cat(y.astype(str), sep=sep),
                  [df[col] for col in cols])

all_data['new_mvn'] = str_join(all_data, '-', 'str_mvn', 'direction', 'type_of_day', 'top_or_middle')


all_data = all_data.rename(columns = {"new_mvn":"mvn"})


all_data = pd.DataFrame(all_data, columns=['route_id','trip_id','survey_id','survey_start_date','survey_end_date',
                                           'stop_name','stop_id','arrival_time','pass_in','pass_out','vehicle_id',
                                           'grafic','trip_num','stop_sequence','fact_trip_date','mvn'])

all_data['arrival_time']=all_data[['arrival_time']].astype(str)

# all_data['mvn']=all_data.mvn.str.replace('/', '_')

all_data

,route_id,trip_id,survey_id,survey_start_date,survey_end_date,stop_name,stop_id,arrival_time,pass_in,pass_out,vehicle_id,grafic,trip_num,stop_sequence,fact_trip_date,mvn
0,162,162,162,2018-11-14,2018-11-14,пос. Обидимо,205,09:48:07,2,0,162,буд,1,1,2018-11-14,162-Обратное-буд-Межпик
1,162,162,162,2018-11-14,2018-11-14,Парковая,216,09:48:39,0,0,162,буд,1,2,2018-11-14,162-Обратное-буд-Межпик
2,162,162,162,2018-11-14,2018-11-14,Больница,219,09:49:11,1,0,162,буд,1,3,2018-11-14,162-Обратное-буд-Межпик
3,162,162,162,2018-11-14,2018-11-14,Микрорайон,223,09:49:43,1,0,162,буд,1,4,2018-11-14,162-Обратное-буд-Межпик
4,162,162,162,2018-11-14,2018-11-14,Совхозная,226,09:50:52,1,0,162,буд,1,5,2018-11-14,162-Обратное-буд-Межпик
5,162,162,162,2018-11-14,2018-11-14,Рынок,235,09:54:28,2,2,162,буд,1,6,2018-11-14,162-Обратное-буд-Межпик
6,162,162,162,2018-11-14,2018-11-14,Каньшино,245,09:55:45.417000,0,0,162,буд,1,7,2018-11-14,162-Обратное-буд-Межпик
7,162,162,162,2018-11-14,2018-11-14,Гараж,249,09:56:31.614000,0,0,162,буд,1,8,2018-11-14,162-Обратное-буд-Межпик
8,162,162,162,2018-11-14,2018-11-14,Шоссейная,254,09:57:38.621000,0,0,162,буд,1,9,2018-11-14,162-Обратное-буд-Межпик
9,162,162,162,2018-11-14,2018-11-14,пос. Барсуки,258,10:01:28,1,0,162,буд,1,10,2018-11-14,162-Обратное-буд-Межпик


In [16]:
# усреднение по максимально встречающейся трассе
count_trips = all_data[['mvn','trip_num','stop_id','stop_sequence']]
count_trips['stop_id']=count_trips[['stop_id']].astype(str)
count_trips['stop_sequence']=count_trips[['stop_sequence']].astype(str)

sequence = count_trips.groupby(['mvn','trip_num'], as_index=False).agg(lambda x: ','.join(x))
sequence['colo'] = 1


sequence_count = sequence.groupby(['mvn','stop_id'])['colo'].agg(['count']).reset_index()

sequence_max = sequence_count.groupby(['mvn'])['count'].agg(['max']).reset_index()

sequence = pd.merge(sequence, sequence_count, how = 'left', on = ['mvn','stop_id'])
sequence = pd.merge(sequence, sequence_max, how = 'left', on = ['mvn'])

sequence = sequence[sequence['count'] == sequence['max']]

sequence

C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,mvn,trip_num,stop_id,stop_sequence,colo,count,max
0,1-Обратное-буд-Вечер пик,2184,"785,757,738,708,710,700,714,720,711,717,669,61...","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",1,2,2
1,1-Обратное-буд-Вечер пик,3607,"785,757,738,708,710,700,714,720,711,717,669,61...","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",1,2,2
3,1-Обратное-буд-Межпик,471,"785,757,738,708,710,700,714,720,711,717,669,61...","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",1,9,9
4,1-Обратное-буд-Межпик,475,"785,757,738,708,710,700,714,720,711,717,669,61...","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",1,9,9
5,1-Обратное-буд-Межпик,2191,"785,757,738,708,710,700,714,720,711,717,669,61...","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",1,9,9
6,1-Обратное-буд-Межпик,3608,"785,757,738,708,710,700,714,720,711,717,669,61...","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",1,9,9
7,1-Обратное-буд-Межпик,3610,"785,757,738,708,710,700,714,720,711,717,669,61...","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",1,9,9
8,1-Обратное-буд-Межпик,4457,"785,757,738,708,710,700,714,720,711,717,669,61...","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",1,9,9
9,1-Обратное-буд-Межпик,4460,"785,757,738,708,710,700,714,720,711,717,669,61...","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",1,9,9
10,1-Обратное-буд-Межпик,4463,"785,757,738,708,710,700,714,720,711,717,669,61...","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",1,9,9


In [17]:
all_data = all_data.merge(sequence[['mvn', 'trip_num']], on = ['mvn', 'trip_num'])
all_data['pass_in_mean'] = all_data.groupby(['mvn', 'stop_id', 'stop_sequence'])['pass_in'].transform('mean')
all_data['pass_out_mean'] = all_data.groupby(['mvn', 'stop_id', 'stop_sequence'])['pass_out'].transform('mean')
del all_data['pass_in']
del all_data['pass_out']
all_data['pass_in'] = all_data['pass_in_mean']
all_data['pass_out'] = all_data['pass_out_mean']
del all_data['pass_in_mean']
del all_data['pass_out_mean']
all_data = all_data.drop_duplicates(['route_id','trip_id','survey_id',
                                           'stop_name','stop_id','pass_in','pass_out','vehicle_id',
                                           'grafic', 'stop_sequence','mvn']).reset_index(drop=True)
all_data

,route_id,trip_id,survey_id,survey_start_date,survey_end_date,stop_name,stop_id,arrival_time,vehicle_id,grafic,trip_num,stop_sequence,fact_trip_date,mvn,pass_in,pass_out
0,162,162,162,2018-11-14,2018-11-14,пос. Обидимо,205,09:48:07,162,буд,1,1,2018-11-14,162-Обратное-буд-Межпик,2.833333,0.000000
1,162,162,162,2018-11-14,2018-11-14,Парковая,216,09:48:39,162,буд,1,2,2018-11-14,162-Обратное-буд-Межпик,1.000000,0.000000
2,162,162,162,2018-11-14,2018-11-14,Больница,219,09:49:11,162,буд,1,3,2018-11-14,162-Обратное-буд-Межпик,1.500000,0.333333
3,162,162,162,2018-11-14,2018-11-14,Микрорайон,223,09:49:43,162,буд,1,4,2018-11-14,162-Обратное-буд-Межпик,0.500000,1.166667
4,162,162,162,2018-11-14,2018-11-14,Совхозная,226,09:50:52,162,буд,1,5,2018-11-14,162-Обратное-буд-Межпик,2.500000,0.166667
5,162,162,162,2018-11-14,2018-11-14,Рынок,235,09:54:28,162,буд,1,6,2018-11-14,162-Обратное-буд-Межпик,1.500000,1.000000
6,162,162,162,2018-11-14,2018-11-14,Каньшино,245,09:55:45.417000,162,буд,1,7,2018-11-14,162-Обратное-буд-Межпик,0.166667,0.000000
7,162,162,162,2018-11-14,2018-11-14,Гараж,249,09:56:31.614000,162,буд,1,8,2018-11-14,162-Обратное-буд-Межпик,0.166667,0.333333
8,162,162,162,2018-11-14,2018-11-14,Шоссейная,254,09:57:38.621000,162,буд,1,9,2018-11-14,162-Обратное-буд-Межпик,0.166667,0.166667
9,162,162,162,2018-11-14,2018-11-14,пос. Барсуки,258,10:01:28,162,буд,1,10,2018-11-14,162-Обратное-буд-Межпик,0.833333,0.666667


In [ ]:
# from datetime import datetime
# ask.to_csv('ask-{:%Y-%m-%d-(%H-%M)}.csv'.format(datetime.now()), sep=';', encoding='cp1251', index=False)

In [18]:
list_mvns = all_data[['mvn']].drop_duplicates('mvn').reset_index(drop=True)
list_mvns

,mvn
0,162-Обратное-буд-Межпик
1,К_169К-Прямое-буд-Утро пик
2,К_169К-Обратное-буд-Утро пик
3,К_162К-Прямое-буд-Утро пик
4,К_162К-Обратное-буд-Утро пик
5,К_162К-Прямое-буд-Межпик
6,К_162К-Обратное-буд-Вечер пик
7,К_162К-Обратное-буд-Межпик
8,К_162К-Прямое-буд-Вечер пик
9,К_161К-Прямое-буд-Утро пик


In [19]:
from datetime import datetime
list_mvns.to_csv('list_mvns-{:%Y-%m-%d-(%H-%M)}.csv'.format(datetime.now()), sep=';', encoding='cp1251', index=False)

In [20]:
for i in tqdm(range(len(list_mvns))):
    list_i = list_mvns.mvn[i]
    df = all_data.copy(deep=True)
    df = df.query('@df.mvn == @list_i')
    df.to_csv('C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/mvns_files/' + list_i + '.csv', sep=';', encoding='cp1251', index=False)


# !!!Для выгрузки файлов по каждому из пиков (утро, межпик, вечер) прописать соответствующее время в директории и условие в цикле!!!

In [21]:
# !!!ПРОПИСАТЬ ЗДЕСЬ!!!
# 16-Обратное-буд-Утро пик
route_name = '16' #  ##- нужно для выгрузки единичного файла, в цикле по всем маршрутам не используется
direction = 'Обратное' # 'Прямое' or 'Обратное' ##- нужно для выгрузки единичного файла, в цикле по всем маршрутам не используется
type_of_day = 'буд' # 'буд' or 'вых'
timing = 'Утро пик' # 'Утро пик' or 'Межпик' or 'Вечер пик'

In [22]:
args = {
    # директория с исходными данными асмпп
    'source' : 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/mvns_files/' + route_name + '-' + direction + '-' + type_of_day + '-' + timing + '.csv',
    
    'all_sources' : 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/mvns_files/',
    
    # справочник выходных и рабочих дней
    'workdays' : 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/workdays.csv',
    # задаем сезоны ММ.ДД-ММ.ДД
    'summer' : '05.01-09.30',
    'winter' : '10.01-04.30',
    # задаем список границ временных интервалов в формате ["ЧЧ:ММ", "ЧЧ:ММ",..., "ЧЧ:ММ"]
    'time' : ['00:00','01:00','02:00','03:00','04:00','05:00','06:00','07:00','08:00','09:00','10:00','11:00',
              '12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00'],
    
    # директория с результатами работы
    'results' : 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/',
    # тип периода
    'period-type' : None,#'лето-раб-06:00-06:59',
    # лог-файл
    'log' : None,
    # коэффициенты весовой фукции min((R/beta)**alpha, 1)
    'alpha' : 2,    
    'beta'  : 1000, # расстояние в км после которого вес равен единице
    'one-stop-dist' : 300, # расстояние между соседними о.п. в км
    # подключение к базе UARMS
    'username' : None,
    'password' : None,
}
args

{'all_sources': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/mvns_files/',
 'alpha': 2,
 'beta': 1000,
 'log': None,
 'one-stop-dist': 300,
 'password': None,
 'period-type': None,
 'results': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/',
 'source': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/mvns_files/16-Обратное-буд-Утро пик.csv',
 'summer': '05.01-09.30',
 'time': ['00:00',
  '01:00',
  '02:00',
  '03:00',
  '04:00',
  '05:00',
  '06:00',
  '07:00',
  '08:00',
  '09:00',
  '10:00',
  '11:00',
  '12:00',
  '13:00',
  '14:00',
  '15:00',
  '16:00',
  '17:00',
  '18:00',
  '19:00',
  '20:00',
  '21:00',
  '22:00',
  '23:00'],
 'username': None,
 'winter': '10.01-04.30',
 'workdays': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/workdays.csv'}

In [23]:
def main_func(args):
    # справочник рабочих дней
    df_workdays = pd.read_csv(args['workdays'], sep=';', date_parser=True)
    df_workdays['dt'] = pd.to_datetime(df_workdays.dt, format="%d.%m.%Y")
    df_workdays['date'] = df_workdays.dt.dt.date
    df_workdays.replace({'workday':{0:'вых',1:'раб'}}, inplace=True)
    
    mvn = args['source'].split('/')[-1][:-4]
    #
    
    
    #______________    
    # загружаем данные асмпп по одному мвн
    df_asmpp = pd.read_csv(args['source'], sep=';', encoding='cp1251')
    # убираем записи имеющие некорректную дату
    #df_asmpp = df_asmpp.query('fact_trip_date' > '20100101')
    
    df_asmpp['date']=df_asmpp[['survey_start_date']].astype(str)
    df_asmpp['date']=df_asmpp.date.str.replace('-', '')
    df_asmpp['date'] = pd.to_numeric(df_asmpp["date"])
    df_asmpp['time']=df_asmpp[['arrival_time']].astype(str)
    
    
    df_asmpp['fact_trip_date'] = pd.to_datetime(df_asmpp['fact_trip_date'])
    
    #
    
    
    #______________    
    # создаем поле ts, в котором указан timestamp прибытия тс на о.п.
    df_asmpp['ts'] = list(map(lambda date, time: pd.datetime(date//10000, # year
                                                             date//100%100, # month 
                                                             date%100, # day
                                                             int(time.split(':')[0]), # hour
                                                             int(time.split(':')[1])), # minutes
                              df_asmpp.date,
                              df_asmpp.time))
    # создаем поле date, в котором указана дата прибытия тс на о.п.
    df_asmpp['date'] = df_asmpp.ts.dt.date
    #
    
    
    #______________
    # корректируем название о.п.
    # не исключена ситуация, что одному stop_id соответсвует несколько значений stop_name
    # выбираем для каждого stop_id одно (последнее по дате) значения stop_name
    if df_asmpp.stop_id.count() > 0:
        df_asmpp.stop_id.fillna(value=0, inplace=True)
        df_stops = df_asmpp.groupby(['stop_id','stop_name']).max()[['ts']].reset_index()
        df_stops = df_stops.groupby(['stop_id']).max()[['ts']].reset_index()\
            .join(df_stops.set_index(['stop_id','ts']), on=['stop_id','ts'])\
            .drop('ts', axis=1)
        # меняем названия о.п. в исходных данных
        df_asmpp = df_asmpp.drop('stop_name', axis=1)\
            .join(df_stops.set_index('stop_id'), on='stop_id')
    else:
        #print('Значение поля stop_id не заполнено')
        #df_asmpp.stop_id.fillna(value=0, inplace=True)
        #df_asmpp['stop_id'] = df_asmpp.stop_name
        df_asmpp['stop_id'] = ''
    #
    
    
    #______________
    # в первую очередь необходимо определить принадлежность данных к одному рейсу
    # в настоящее время рейсы, начавшие движение в конце суток, а закончившие в начале следующих суток, разбиваются на два рейса
    # для второй половины таких рейсов требуется корректировка значений поля date

    # как оказалось grafic также может быть запонен с ошибками. при обработке данных за 201709 40 мвн имеют более 2 вариантов трассы
    # можно попробовать дополнительно к grafic брать последние три цифры из vehicle_id
    df_asmpp['veh_id'] = df_asmpp.vehicle_id.astype(str).str[-3:]
    # как оказалось время прибытия тс на о.п. может быть указано с погрешностью 2-3 минуты, т.е. не исключена ситуация, 
    # когда на следующий о.п. тс приезжает после того как была на предыдущем

    # пока размышляю что с этим делать, есть ощущение, что необходимо ориентироваться на порядок строк в апи
    # возможно следует вренуться к этой проблеме после обработки все данных из апи

    # сортируем данные по времени прибытия тс на о.п.
    df_asmpp.sort_values(['grafic', 'veh_id', 'ts','trip_num','stop_sequence'], inplace=True)
    #df_asmpp.sort_values(['date','grafic','trip_num','stop_sequence'], inplace=True)
    # убираем дубли
    df_asmpp.drop_duplicates(subset=['grafic','veh_id','ts','stop_sequence'], inplace=True)
    # определяем порядковый номер предыдущего о.п. для тс
    df_asmpp['stop_seq_prev'] = df_asmpp.groupby('grafic').shift(1).stop_sequence
    # если порядковый номер предыдущего о.п. больше либо равен чем порядковый номер текущего о.п., то начинаем новый рейс
    df_asmpp['race_id'] = 0
    df_asmpp.loc[~(df_asmpp.stop_seq_prev < df_asmpp.stop_sequence), 'race_id'] = 1
    df_asmpp['race_id'] = df_asmpp.race_id.cumsum()
    # убираем техническое поле
    df_asmpp.drop('stop_seq_prev', axis=1, inplace=True)
    # корректируем значение поля fact_trip_date
    df_asmpp = df_asmpp.drop('date', axis=1).join(df_asmpp.groupby('race_id').min()['date'], on='race_id')
    
    
    # находим всевозможные варианты поостановочной трассы
    df_asmpp['trace'] = df_asmpp.stop_sequence.astype(str) + '-' + df_asmpp.stop_id.astype(str) + ',\n'
#     df_asmpp = df_asmpp.drop('trace', axis=1)\
#                        .join(df_asmpp.groupby('race_id').apply(lambda x: x.astype(str).sum())[['trace']], on='race_id')
#     df_asmpp['trace'] = df_asmpp.trace.str[:-2]
    #print('Число различных вариантов поостановочной трассы -- %d.' % len(df_asmpp['trace'].unique()))
    #
    
    
    #______________
    # формируем таблицу рейсов в которой содержится информация о дате и времени прибытия каждого рейса на первый о.п.
    df_races = df_asmpp.groupby('race_id').min()[['trip_id','trace','date','ts']].reset_index()
    # добавляем число пассажиров
    df_races = df_races.join(df_asmpp.groupby('race_id').sum()[['pass_in','pass_out']], on=['race_id'])
    df_races['pass'] = (df_races.pass_in + df_races.pass_out)/2
    df_races.drop(['pass_in','pass_out'], axis=1, inplace=True)

    # для каждого рейса определяем тип временного периода

    # определяем принадлежность рейса к рабочему или выходному дню
    df_races = df_races.join(df_workdays.set_index('date')['workday'], on='date')

    #--------------
    # определяем принадлежность рейса к сезону
    df_races['season'] = 'неоп'
    # формируем даты начала и окончания летнего сезона для каждого рейса
    df_races['start'] = list(map(lambda t: pd.datetime(t.year, 
                                                       int(args['summer'][:2]),
                                                       int(args['summer'][3:5])), df_races.ts))
    df_races['end']   = list(map(lambda t: pd.datetime(t.year, 
                                                       int(args['summer'][6:8]),
                                                       int(args['summer'][9:11])), df_races.ts))
    # определяем принадлежность рейса к летнему сезону
    df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'season'] = 'лето'
    # формируем первый вариант даты начала и окончания зимнего сезона для каждого рейса
    df_races['start'] = list(map(lambda t: pd.datetime(t.year, 
                                                       int(args['winter'][:2]),
                                                       int(args['winter'][3:5])), df_races.ts))
    df_races['end']   = list(map(lambda t: pd.datetime(t.year + 1, 
                                                       int(args['winter'][6:8]),
                                                       int(args['winter'][9:11])), df_races.ts))
    # определяем принадлежность рейса к зимнему сезону
    df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'season'] = 'зима'
    # формируем второй вариант даты начала и окончания зимнего сезона для каждого рейса
    df_races['start'] = list(map(lambda t: pd.datetime(t.year - 1, 
                                                       int(args['winter'][:2]),
                                                       int(args['winter'][3:5])), df_races.ts))
    df_races['end']   = list(map(lambda t: pd.datetime(t.year, 
                                                       int(args['winter'][6:8]),
                                                       int(args['winter'][9:11])), df_races.ts))
    # определяем принадлежность рейса к зимнему сезону
    df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'season'] = 'зима'
    # убираем технические поля
    df_races.drop(['start','end'], axis=1, inplace=True)
    #---------------

    # определяем принадлежность рейса к часу суток
    df_races['time'] = ''
    for i in range(len(args['time'])):
        start = args['time'][i-1]
        end   = args['time'][i]
        end = (pd.datetime(2000,1,1,int(end[:2]),int(end[-2:])) - pd.Timedelta(value=1, unit='m')).strftime('%H:%M')
        time = '%s-%s' % (start, end)
        if start < end:
            df_races['start'] = list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(start[:2]),int(start[-2:])), df_races.ts))
            df_races['end'] =   list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(end[:2]),int(end[-2:])), df_races.ts))
            df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'time'] = time
        else:
            df_races['start'] = list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(start[:2]),int(start[-2:])), df_races.ts))
            df_races['end'] =   list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(end[:2]),int(end[-2:]))\
                                                 + pd.Timedelta(value=1, unit='d'), df_races.ts))
            df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'time'] = time
            df_races['start'] = list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(start[:2]),int(start[-2:]))\
                                                - pd.Timedelta(value=1, unit='d'), df_races.ts))
            df_races['end'] =   list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(end[:2]),int(end[-2:])), df_races.ts))
            df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'time'] = time

    # задаем тип временного интервала
    df_races['period_type'] = df_races.season.astype(str) + '-' +  df_races.workday.astype(str) + '-' + df_races.time.astype(str)
    #
    
    
    #______________
    # формируем таблицу вариантов поостановочных трасс

    # определяем даты, за которые есть данные асмпп по каждому из вариантов трассы и число рейсов
    df_traces =\
        df_races.groupby(['trace']).min()[['date']]\
        .join(df_races.groupby(['trace']).max()[['date']], lsuffix='_first', rsuffix='_last')\
        .join(df_races.groupby(['trace']).count()[['race_id']])\
        .rename(columns={'race_id':'n_races'})\
        .sort_values(['date_last','n_races'], ascending=False)\
        .reset_index()
    # создаем идентификатор варианта трассы
    df_traces['trace_id'] = range(1, len(df_traces) + 1)
    # добавляем идентификатор трассы в таблицу рейсов
    df_races = df_races.join(df_traces.set_index('trace')['trace_id'], on='trace').drop('trace', axis=1)
    # добавляем информацию о том как соотносятся между собой trace_id и trip_id 
    df_tmp = df_races[['trip_id','trace_id']].drop_duplicates().set_index('trace_id').sort_values('trip_id')
    df_tmp['trip_id'] = df_tmp.trip_id.astype(str) + ','
    df_traces = df_traces.join(df_tmp.groupby(level='trace_id').apply(lambda x: x.astype(str).sum()), on='trace_id')
    df_traces['trip_id'] = df_traces.trip_id.str[:-1]
    # создаем поле, в котором будет отмечено следует ли использовать указанный вариант поостановочной трассы для построения матрицы
    df_traces['flag'] = 0
    # Определяем приоритетный вариант трассы по которому будет строиться матрица из следующих соображений:
    # 1. Фиксируется варинат трассы, по которому имеются наиболее свежие данные асмпп (при наличии нескольких таких вариантов 
    #    трассы, выбирается тот, у которого наибольшее число рейсов). Для зафиксированного варианта определяется дата первого
    #    обследования.
    # 2. Выделяются все варианты трассы, по которым дата последнего обследования более поздняя, чем зафикисрованная дата.
    # 3. Из них выбирается вариант трассы с наибольшим числом обследований, если таких вариантов несколько,
    #    то выбирается один вариант случайным образом.
    date_first = df_traces.sort_values(['date_last','n_races'], ascending=False).iloc[0].date_first
    mask = df_traces.date_last >= date_first
    n_races = df_traces[mask].n_races.max()
    df_traces.loc[(mask) & (df_traces.n_races == n_races), 'flag'] = 1
    #
    
    
    #______________
    # определяем число рейсов по каждому из вариантов трассы за каждый тип временного интервала
    df_nraces = df_races.groupby(['trace_id','season','workday','time']).count()['race_id']\
        .unstack('trace_id')\
        .fillna(value=0)\
        .astype(int)\
        .reset_index()\
        .sort_values(['season','workday','time'], ascending=[True,False,True])
    #
    
    
    #______________
    # определяем число пассажиров, перевезенных средним рейсом по каждому из вариантов трассы за каждый тип временного интервала
    df_pass =(\
        df_races.groupby(['trace_id','season','workday','time']).mean()['pass']\
            .unstack('trace_id')\
            .fillna(value=0)\
            .round(1)\
            .astype(str)\
        + ' ± ' +\
        df_races.groupby(['trace_id','season','workday','time']).std()['pass']\
            .unstack('trace_id')\
            .fillna(value=0)\
            .round(1)\
            .astype(str)\
        ).reset_index()\
        .sort_values(['season','workday','time'], ascending=[True,False,True])
    #
    
    
    #______________
    # вычисляем посадку и высадку среднего рейса по всем типам временных интервалов и вариантам поостановочной трассы
    df_inout =\
        df_asmpp[['mvn','race_id','stop_sequence','stop_id','stop_name','pass_in','pass_out']]\
        .join(df_races.set_index('race_id')[['trace_id','period_type']], on='race_id')\
        .groupby(['mvn','trace_id','period_type','stop_sequence','stop_id','stop_name']).mean()[['pass_in','pass_out']]\
        .reset_index()
    #
    
    
    #______________
    # формируем xlsx файл с результатами предобработки
    xls_writer = pd.ExcelWriter(args['results'] + '/'+ mvn + '.xlsx')

    # сохраняем вкладку inout
    df_inout.to_excel(xls_writer, index=False, sheet_name='inout')
    xls_sheet = xls_writer.sheets['inout']
    xls_sheet.set_column('A:A', len(mvn) + 1, None)
    xls_sheet.set_column('C:C', 21, None)
    xls_sheet.set_column('F:F', (df_inout.stop_name.str.len()).max() + 1, None)
    xls_fmt = xls_writer.book.add_format(properties={'num_format': '0.00'})
    xls_sheet.set_column('G:H', None, xls_fmt)

    # сохраняем вкладку traces
    df_traces.set_index('trace_id')[['n_races','date_first','date_last','flag','trip_id','trace']]\
        .to_excel(xls_writer, sheet_name='traces')
    xls_sheet = xls_writer.sheets['traces']
    xls_sheet.set_column('C:D', 11, None)

    # сохраняем вкладку nraces
    df_nraces.to_excel(xls_writer, index=False, sheet_name='nraces')
    xls_sheet = xls_writer.sheets['nraces']
    xls_sheet.set_column('C:C', 11, None)
    xls_sheet.set_column('D:ZZ', 4, None)

    # сохраняем вкладку pass
    df_pass.to_excel(xls_writer, index=False, sheet_name='pass')
    xls_sheet = xls_writer.sheets['pass']
    xls_sheet.set_column('C:C', 11, None)
    xls_sheet.set_column('D:ZZ', 14, None)

    # сохраняем и закрываем файл
    xls_writer.save()
    #
    #
    return len(df_traces)

In [24]:
call_func = main_func(args)
call_func

1

# Part B2

In [25]:
# запускаем расчет для всех файлов в директории

ts_start = pd.datetime.now()
lst_mvns = list()
lst_ntraces = list()
path = args['all_sources']
for file in tqdm(os.listdir(path=path)):
    if file.split('.')[-1] == 'csv':
        args['source'] = path + '/' + file
        mvn = file[:-4]
        lst_mvns.append(mvn)
        try:
            n = main_func(args)
            lst_ntraces.append(n)
        except:
            print('!!! ВНИМАНИЕ !!! при обработке данных возникла ошибка')
            lst_ntraces.append('ERROR')
# 

# print(lst_mvns)

In [26]:
df_results = pd.DataFrame({'mvn': lst_mvns, 'ntraces': lst_ntraces})
df_results.to_csv(args['results'] + '/log' + ts_start.strftime('-%Y-%m-%d-%H-%M')+ '.csv', sep=';', index=False)

# Part C1

In [ ]:
# 'source' : 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/mvns_files/' + route_name + '-' + direction + '-' + type_of_day + '-' + timing + '.csv',
    
# experiment = pd.read_excel('C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/преобразованные '  
#                            + route_name + '-' + direction + '-' + type_of_day + '-' + timing + '.xlsx')
# experiment

In [27]:
# задаем директорию с исходными данными
args['source'] = 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/2-Обратное-буд-Утро пик.xlsx'
path = args['source']


# !!!!!!
# задаем директорию с результатами
args['results'] = ('C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/преобразованные '
+ timing +'/' + type_of_day +'/')


# логин и пароль для подключения к базе UARMS
args['username'] = None
args['password'] = None
args['period-type'] = None
args

{'all_sources': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/mvns_files/',
 'alpha': 2,
 'beta': 1000,
 'log': None,
 'one-stop-dist': 300,
 'password': None,
 'period-type': None,
 'results': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/преобразованные Утро пик/буд/',
 'source': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/2-Обратное-буд-Утро пик.xlsx',
 'summer': '05.01-09.30',
 'time': ['00:00',
  '01:00',
  '02:00',
  '03:00',
  '04:00',
  '05:00',
  '06:00',
  '07:00',
  '08:00',
  '09:00',
  '10:00',
  '11:00',
  '12:00',
  '13:00',
  '14:00',
  '15:00',
  '16:00',
  '17:00',
  '18:00',
  '19:00',
  '20:00',
  '21:00',
  '22:00',
  '23:00'],
 'username': None,
 'winter': '10.01-04.30',
 'workdays': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/workdays.csv'}

In [28]:
def main_args_C1(args):
#     args = args_C1
    # открываем лог-файл
    if args['log'] is not None:
        outstream = open('%s/%s' % (args['results'], args['log']), mode='a') 
    else:
        outstream = None
    # выводим информацию о запуске расчета
    print('%s -- запуск расчета с параметрами:' % pd.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), file=outstream)
    for arg in args:
        if (arg != 'username') & (arg != 'password'):
            print('%15s : %s' % (arg, args[arg]), file=outstream)
    #
    # определяем мвн
    mvn = args['source'].split('/')[-1].split('.')[0]
    #
    
    
    #______________
    # загружаем данные о поостановочных трассах, которые необходимо использовать для получения посадки-высадки
    print('Загружаем данные о вариантах поостановочных трасс...', file=outstream)
    try:
        df_traces = pd.read_excel(args['source'], sheetname='traces')
        print('...общее число вариантов поостановочных трасс -- %d...' % len(df_traces), file=outstream)
        try:
            lst_traces = list(df_traces.query('flag == 1').trace_id.unique())
            if len(lst_traces) == 0:
                print('...варианты поостановочных трассы для построения матрицы не выбраны...', file=outstream)
                print('...используем все варианты поостановочных трасс.', file=outstream)    
            else:
                print('...число выбранных вариaнтов поостановочной трассы -- %d.' % len(lst_traces), file=outstream)
        except:
            lst_traces = list()
    except:
        df_traces = None
        lst_traces = list()
        print('...нет информации о вариантах поостановочных трасс.', file=outstream)
    #
    #
    
    
    #______________
    #transpose_stop_id = df_traces['trace'][0]
    #вводим переменную transpose_stop_id, в которую записываем ячейку trace с последовательностью остановок
    #list_transpose = transpose_stop_id.split(',\n')
    #list_transpose
    #list_unstack = list_transpose.unstack()
    #
    
    
    #______________
    # читаем данные о посадке и высадке
    df_asmpp = pd.read_excel(args['source'], sheetname='inout',)# dtype={'pass_in':float,'pass_out':float})
    nasmpp_all = len(df_asmpp)
    # выбираем данные за требуемый период
    #df_asmpp = df_asmpp.query('period_type == "%s"' % (args['period-type']))
    nasmpp_per = len(df_asmpp)
    # выбираем помаршрутные трассы
    df_asmpp = df_asmpp[df_asmpp.trace_id.isin(lst_traces)]
    if len(df_asmpp) > 0:
        print('Поостановная трасса для построения матрицы:', file=outstream)
        # формируем таблицу с названием о.п. на случай если помаршрутные трассы отличаются
        df_stops = df_asmpp[['stop_sequence','stop_id','stop_name']]\
            .drop_duplicates(subset=['stop_sequence']).sort_values('stop_sequence')
        print(df_stops.set_index('stop_sequence'), file=outstream)
        # определяем число рейсов по каждому из вариантов поостановочной трассы
        print('Загружаем данные о числе рейсов по указанным вариантам поостановочной трассы...', file=outstream)
        try:
            # читаем данные о посадке и высадке за требуемый период
            df_nraces = pd.read_excel(args['source'], sheetname='nraces')\
                .query('season == "%s" and workday == "%s" and time == "%s-%s"' % tuple(args['period-type'].split('-')))\
                .set_index(['season','workday','time'])\
                .stack()\
                .reset_index()\
                .rename(columns={'level_3':'trace_id',0:'nraces'})
            # выбираем только указанные варианты поостановочной трассы
            df_nraces = df_nraces[df_nraces.trace_id.isin(lst_traces)]
            # добавляем в таблицу с данными асмпп информацию о числе рейсов
            df_asmpp = df_asmpp.join(df_nraces.set_index('trace_id')['nraces'], on='trace_id')
            # вычисляем посадку и высадку с учетом числа рейсов
            df_asmpp['pass_in']  = df_asmpp.eval('pass_in  * nraces')
            df_asmpp['pass_out'] = df_asmpp.eval('pass_out * nraces')
            # определяем число рейсов
            nraces = df_nraces.nraces.sum()
            print('...число рейсов -- %d.' % nraces, file=outstream)
        except:
            print('...нет данных о числе рейсов....', file=outstream)
            print('...считаем, что по каждому варианту был совершен один рейс.', file=outstream)
            nraces = len(lst_traces)
        # определяем исходные посадку и высадку
        print('Загружаем исходные данные о посадке-высадке:', file=outstream)
        df_asmpp = df_stops.join(df_asmpp.groupby('stop_sequence').sum()[['pass_in','pass_out']]/nraces, on='stop_sequence')
        print(df_asmpp.set_index('stop_sequence')[['stop_id','pass_in','pass_out']].round(1), file=outstream)
        print('Суммарно за рейс:', file=outstream)
        print('pass_in  %6.1f' % df_asmpp.pass_in.sum(), file=outstream)
        print('pass_out %6.1f' % df_asmpp.pass_out.sum(), file=outstream)
    else:
        nraces = 0
        df_stops = None
        print('Загружаем исходные данные о посадке-высадке...', file=outstream)
        print('...общее число записей в данных асмпп -- %d...' % nasmpp_all, file=outstream)
        if nasmpp_per > 0:
            print('...число записей в данных асмпп за указанный период -- %d...' % nasmpp_per, file=outstream)
            print('...за указанный период отсутствуют данные по выбранным вариантам поостановочной трассы.', file=outstream)
        else:
            print('...за указанный период отсутствуют данные асмпп.', file=outstream)
    #
    #
    
    
    #______________
    if len(df_asmpp) > 0:
        print('Вносим поправки в исходные данные для корректного вычисления матрицы...', file=outstream)
        # вносим корректировки с данные о посадке и высадке для корректного вычисления матрицы
        df_inout = df_asmpp.copy().set_index('stop_sequence')
        # определяем порядковый номер первого о.п.
        ss_min = df_asmpp.stop_sequence.min()
        print('...наименьший порядковый номер о.п. в поостановочной трассе -- %d...' % ss_min, file=outstream)
        # обнуляем высадку на первом о.п.
        if df_inout.loc[ss_min, 'pass_out'] != 0:
            print('...высадка на начальном о.п. составляет %.1f -- приравниваем нулю...' % df_inout.loc[ss_min, 'pass_out'],
                  file=outstream)
            df_inout.loc[ss_min, 'pass_out'] = 0
        # определяем порядковый номер последнего о.п.
        ss_max = df_asmpp.stop_sequence.max()
        print('...наибольший порядковый номер о.п. в поостановочной трассе -- %d...' % ss_max, file=outstream)
        # обнуляем посадку на последнем о.п.
        if df_inout.loc[ss_max, 'pass_in'] != 0:
            print('...посадка на последнем о.п. составляет %.1f -- приравниваем нулю...' % df_inout.loc[ss_max, 'pass_in'],
                  file=outstream)
            df_inout.loc[ss_max, 'pass_in'] = 0
        # вычисляем пассажиропоток на перегонах
        df_cumsum = df_inout[['pass_in', 'pass_out']].cumsum()
        df_cumsum['pass_inside'] = df_cumsum.eval('pass_in - pass_out')
        df_inout = df_inout.join(df_cumsum[['pass_inside']])
        # корректируем посадку на первом о.п., при необходимости
        diff = (df_inout.pass_inside - df_inout.pass_in).min()
        if diff < 0:
            print('...зафиксировано %.1f пассажиров в салоне тс, увеличиваем посадку на начальном о.п...' % diff, file=outstream)
            df_inout.loc[ss_min, 'pass_in'] = df_inout.loc[ss_min, 'pass_in'] - diff
            df_inout['pass_inside'] = df_inout['pass_inside'] - diff
        # корректируем высадку на последнем о.п., при необходимости
        if df_inout.loc[ss_max, 'pass_inside'] > 0:
            print('...зафиксировано %.1f пассажиров в салоне тс по окончании рейса, увеличиваем высадку на последнем о.п...'
                      % df_inout.loc[ss_max, 'pass_inside'], file=outstream)
            df_inout.loc[ss_max, 'pass_out'] = df_inout.loc[ss_max, 'pass_out'] + df_inout.loc[ss_max, 'pass_inside']
            df_inout.loc[ss_max, 'pass_inside'] = 0
        df_inout.reset_index(inplace=True)
        print('...корректрировка выполнена:', file=outstream)
        print(df_inout.set_index('stop_sequence')[['stop_id','pass_in','pass_out']].round(1), file=outstream)
        print('Суммарно за рейс:', file=outstream)
        print('pass_in  %6.1f' % df_inout.pass_in.sum(), file=outstream)
        print('pass_out %6.1f' % df_inout.pass_out.sum(), file=outstream)
    else:
        df_inout = None
    #
    #
    
    
    #______________
    # подгружаем данные о координатах о.п.
    if df_stops is not None:
        print('Загружаем данные о местоположении о.п...', file=outstream)
        # формируем список уникальных идентификаторов о.п.
        lst_stops = list(df_stops.stop_id.dropna().unique())
        if len(lst_stops) > 0:
            print('...для определения местоположения о.п. подключаемся к базе UARMS...', file=outstream)
            try:
                # инициализируем подключение к базе данных
                cnx = create_engine('postgresql://%s:%s@airms.mgtniip.ru:5432/UARMS'
                    % (args['username'], args['password']))
                # загружаем данные из базы
                gdf_stops = gpd.read_postgis(\
                    "SELECT stop_id,geometry FROM routes.stops_latest as stops WHERE stops.stop_id IN%s" \
                    % (str(tuple(lst_stops)) if len(lst_stops) > 1 else '(' + str(lst_stops[0]) + ')'),\
                    con=cnx, geom_col='geometry')
                if len(gdf_stops) > 0:
                    print('...загружены данные о местоположении %d уникальных о.п.' % len(gdf_stops), file=outstream)
                    # задаем проекцию и проецируем точки на плоскость для вычисляения расстояний по прямой
                    gdf_stops.crs = from_epsg(4326)
                    gdf_stops = gdf_stops.to_crs(from_epsg(32637))
                    # создаем поля с координатами о.п.
                    gdf_stops[['x','y']] = gdf_stops.geometry.bounds[['minx','miny']]
                    gdf_stops.drop('geometry', axis=1, inplace=True)
                    gdf_stops.set_index('stop_id', inplace=True)
                else:
                    print('...в базе нет информации о местоположении ни одного о.п.', file=outstream)
            except:
                print('...не могу загрузить данные из базы -- отсутсвует соединение или неверные логин/пароль.', file=outstream)
                gdf_stops = None
        else:
            print('...в поостановочной трассе отсутствует информация об идентификаторах о.п.', file=outstream)
            gdf_stops = None
    #
    #
    
    
    #______________
    # программная реализация прохода "с двух сторон"
    if df_inout is not None:
        print('Запускаем процедуру вычисления маршрутной матрицы корреспонденций...', file=outstream)  
        print('...инициализация матриц...', file=outstream)  
        # инициализируем матрицы
        start_stop_name = []
        start_stop_id = []
        start_stop_ss = []
        end_stop_name = []
        end_stop_id = []
        end_stop_ss = []
        weight = []
        corr_value = []
        for i, i_id, i_name in zip(df_inout.stop_sequence, df_inout.stop_id, df_inout.stop_name):
            for j, j_id, j_name in zip(df_inout.stop_sequence, df_inout.stop_id, df_inout.stop_name):
                start_stop_ss.append(i)
                start_stop_id.append(i_id)
                end_stop_ss.append(j)
                end_stop_id.append(j_id)
                start_stop_name.append(i_name)
                end_stop_name.append(j_name)
                # инициализируем маршрутную матрицу корреспонденций
                corr_value.append(np.nan)
                # инициализируем матрицу весов
                # определяем расстояние между о.п.
                try:
                    stop2stop_distance =\
                        ( (gdf_stops.loc[i, 'x'] - gdf_stops.loc[j, 'x'])**2\
                        + (gdf_stops.loc[i, 'y'] - gdf_stops.loc[j, 'y'])**2)**0.5  
                except:
                    stop2stop_distance = abs(j-i)*args['one-stop-dist']
                weight.append(min((stop2stop_distance/args['beta'])**args['alpha'], 1))
        df_matrix = pd.DataFrame({'start_num': start_stop_ss,
                                  'start_id' : start_stop_id,
                                  'start_name': start_stop_name,
                                  'end_num': end_stop_ss,
                                  'end_id' : end_stop_id,
                                  'end_name': end_stop_name,
                                  'weight' : weight,
                                  'value': corr_value}).set_index(['start_num','end_num'])
        print('...инициализация векторов...', file=outstream)  
        # инициализация векторов
        inleft = df_inout.set_index('stop_sequence')['pass_in']
        outleft = df_inout.set_index('stop_sequence')['pass_out']
        print('...инициализация переменных...', file=outstream)  
        # инициализация переменных
        lst_ss = list(df_inout.stop_sequence)
        #n = len(lst_ss)
        i = lst_ss[-1]
        # шаг интерации
        print('...расчет матрицы...', file=outstream)  
        for j in lst_ss[1:]:
            # первый этап - распределяем высадку на j-ом о.п.
            # определяем индекс порядкового номера о.п., до которого идет распределение посадки
            m = min(lst_ss.index(j)-1, lst_ss.index(i)-1)
            # опредеяем максимальный вес
            try:
                w_max = df_matrix.query('start_num <= %d and end_num == %d' % (lst_ss[m], j)).weight.max()
            except:
                w_max = 1
            # вычисляем эффективную высадку
            eff = 0
            inleftsum = inleft.loc[:lst_ss[m]].sum()
            for k in lst_ss[:(m+1)]:
                w = df_matrix.loc[(k,j),'weight']
                df_matrix.loc[(k,j),'value'] = (outleft[j] * (w / w_max) * (inleft[k] / inleftsum)) if (inleftsum > 0) else 0
                eff = eff + df_matrix.loc[(k,j),'value']
            # нормируем высадку
            for k in lst_ss[:(m+1)]:
                x = df_matrix.loc[(k,j),'value'] * outleft[j] / eff if eff > 0 else 0
                # проверяем что высадка не превосходит нераспределенную посадку
                if x > inleft[k]:
                    x = inleft[k]
                # обновляем нераспределенную посадку и высадку
                inleft[k] = inleft[k] - x
                outleft[j] = outleft[j] - x
                df_matrix.loc[(k,j),'value'] = x
                # пересчитываем эффективную высадку
                eff = 0
                for l in lst_ss[(lst_ss.index(k)+1):(m+1)]:
                    eff = eff + df_matrix.loc[(l,j),'value']
            # ----------------------
            # второй этап итерации - распределяем посадку i-ого о.п.
            i = lst_ss[-1-lst_ss.index(j)]
            # определяем индекс порядкового номера о.п., с которого идет распределение высадки
            m = max(lst_ss.index(j)+1, lst_ss.index(i)+1)
            # опредеяем максимальный вес
            try:
                w_max = df_matrix.query('start_num == %d and end_num >= %d' % (i, lst_ss[m])).weight.max()
            except:
                w_max = 1
            # вычисляем эффективную высадку
            eff = 0
            outleftsum = outleft.loc[lst_ss[m-1]:].sum()
            for k in lst_ss[:(m-1):-1]:
                w = df_matrix.loc[(i,k),'weight']
                df_matrix.loc[(i,k),'value'] = (inleft[i] * (w / w_max) * (outleft[k] / outleftsum)) if (outleftsum > 0) else 0
                eff = eff + df_matrix.loc[(i,k),'value']
            # нормируем высадку
            for k in lst_ss[:(m-1):-1]:
                x = df_matrix.loc[(i,k),'value'] * inleft[i] / eff if eff > 0 else 0
                # проверяем что высадка не превосходит нераспределенную посадку
                if x > outleft[k]:
                    x = outleft[k]
                # обновляем нераспределенную посадку и высадку
                inleft[i] = inleft[i] - x
                outleft[k] = outleft[k] - x
                df_matrix.loc[(i,k),'value'] = x
                # пересчитываем эффективную высадку
                eff = 0
                for l in lst_ss[(lst_ss.index(k)-1):(m-1):-1]:
                    eff = eff + df_matrix.loc[(i,l),'value']  
            # пишем лог
            print('...о.п. отпр. %3d -- о.п. приб. %3d -- нераспр. посадка %6.2f -- нераспр. высадка %6.2f...' 
                  % (i, j, inleft.sum(), outleft.sum()), file=outstream)
        df_matrix.reset_index(inplace=True)
        df_matrix = df_matrix.drop('weight', axis=1)
        print('...расчет выполнен.', file=outstream)
    else:
        df_matrix = None
    #
    #
    
    
    #______________
    if df_matrix is not None:
        print('Проверяем контрольные суммы по посадке и высадке...', file=outstream)
        # проверяем, что поостановочная посадка и высадка совпадают
        df_compare = df_inout.join(df_matrix.groupby('start_num').sum()[['value']], on='stop_sequence')\
            .join(df_matrix.groupby('end_num').sum()[['value']], on='stop_sequence', lsuffix='_ent', rsuffix='_ext')\
            .fillna(value=0).round(2)

        df_diff_ent = df_compare[df_compare.pass_in  != df_compare.value_ent]
        df_diff_ext = df_compare[df_compare.pass_out != df_compare.value_ext]
        if len(df_diff_ent) > 0:
            print('...данные по посадке отличаются на следующих о.п.:', file=outstream)
            print(df_diff_ent.set_index('stop_sequence')[['pass_in','value_ent']], file=outstream)
        if len(df_diff_ext) > 0:
            print('...данные по высадке отличаются на следующих о.п.:', file=outstream)
            print(df_diff_ext.set_index('stop_sequence')[['pass_out','value_ext']], file=outstream)
        if len(df_diff_ent) + len(df_diff_ext) == 0:
            print('...поостановочные данные по посадке и высадке совпадают.', file=outstream)
    #
    #
    
    
    #______________
    if df_matrix is not None:
        # подготавливаем таблицу для экспорта
        df_table = \
        df_matrix.fillna(value='').rename(columns={'start_name':'Наименование о.п.', 'value':'# о.п. приб.',\
        'start_num':'# о.п. отпр.', 'end_num':mvn, 'start_id':'Идентиф. # о.п.'})\
        .set_index(['# о.п. отпр.', 'Идентиф. # о.п.', 'Наименование о.п.', mvn])[['# о.п. приб.']].unstack()
    #
    #
    
    
    #______________
    # меняем формат значений идентификаторов о.п.
    for fld in ['start_id', 'end_id']:
        df_matrix[fld] = df_matrix[fld].fillna(value=-1).astype(int).replace(-1,'')
    df_inout['stop_id'] = df_inout['stop_id'].fillna(value=-1).astype(int).replace(-1,'')
    df_asmpp['stop_id'] = df_asmpp['stop_id'].fillna(value=-1).astype(int).replace(-1,'')
    #
    
    
    #______________
    if df_matrix is not None:
        print('Сохраняем результаты расчетов в файл %s/%s.xlsx...' % (args['results'], mvn), file=outstream)
        try:
            xls_writer = pd.ExcelWriter(args['results'] + '/'+ mvn + '.xlsx')
            # сохраняем вкладку исх-посадка-высадка
            print('...создаем вкладку "исх-посадка-высадка"...', file=outstream)
            df_asmpp[['stop_id','stop_name','stop_sequence','pass_in','pass_out']]\
                .fillna(value='')\
                .rename(columns={'stop_name':'Наименование о.п.', 'stop_sequence':'# о.п.',\
                                 'pass_in':'Посадка','pass_out':'Высадка','stop_id':'Идентиф. # о.п.'})\
                .set_index(['# о.п.', 'Идентиф. # о.п.', 'Наименование о.п.'])\
                .to_excel(xls_writer, sheet_name='исх-посадка-высадка')
            xls_sheet = xls_writer.sheets['исх-посадка-высадка']
            xls_fmt2 = xls_writer.book.add_format(properties={'num_format': '0.0'})
            xls_sheet.set_column('D:F', 9, xls_fmt2)
            xls_sheet.set_column('C:C', df_matrix.start_name.str.len().max()+4, None)
            xls_sheet.set_column('B:B', 18, None)
            # сохраняем вкладку посадка-высадка
            print('...создаем вкладку "посадка-высадка"...', file=outstream)
            df_inout[['stop_id','stop_name','stop_sequence','pass_in','pass_out']]\
                .fillna(value='')\
                .rename(columns={'stop_name':'Наименование о.п.', 'stop_sequence':'# о.п.',\
                                 'pass_in':'Посадка','pass_out':'Высадка','stop_id':'Идентиф. # о.п.'})\
                .set_index(['# о.п.', 'Идентиф. # о.п.', 'Наименование о.п.'])\
                .to_excel(xls_writer, sheet_name='посадка-высадка')
            xls_sheet = xls_writer.sheets['посадка-высадка']
            xls_fmt2 = xls_writer.book.add_format(properties={'num_format': '0.0'})
            xls_sheet.set_column('D:F', 9, xls_fmt2)
            xls_sheet.set_column('C:C', df_matrix.start_name.str.len().max()+4, None)
            xls_sheet.set_column('B:B', 18, None)
            # сохраняем вкладку матрица
            print('...создаем вкладку "матрица"...', file=outstream)
            df_table.to_excel(xls_writer, sheet_name='матрица')
            xls_sheet = xls_writer.sheets['матрица']
            xls_fmt = xls_writer.book.add_format(properties={'num_format': '0.000'})
            xls_sheet.set_column('A:ZZ', 7, xls_fmt)
            xls_sheet.set_column('A:A', 12, None)
            xls_sheet.set_column('B:B', 18, None)
            xls_sheet.set_column('C:C', df_matrix.start_name.str.len().max()+4, None)
            # сохраняем и закрываем файл
            xls_writer.save()
            print('...сохранение результатов завершено.', file=outstream)
        except:
            print('...при записи файла возникла ошибка!', file=outstream)
    #
    #
    
    
    #______________
    # закрываем лог-файл
    if args['log'] is not None:
        print('----------------------------------------------------------------------------------------------------',
              file=outstream)
        outstream.close()
    #

    return (nraces, df_asmpp, df_matrix)

In [29]:
call_func_C1 = main_args_C1(args)
call_func_C1

2019-05-29 10:11:32 -- запуск расчета с параметрами:
         source : C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/2-Обратное-буд-Утро пик.xlsx
    all_sources : C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/mvns_files/
       workdays : C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/workdays.csv
         summer : 05.01-09.30
         winter : 10.01-04.30
           time : ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']
        results : C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/преобразованные Утро пик/буд/
    period-type : None
            log : None
          alpha : 2
           beta : 1000
  one-stop-dist : 300
Загружаем данные о вариантах поостановочных трасс...


C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


...общее число вариантов поостановочных трасс -- 1...
...число выбранных вариaнтов поостановочной трассы -- 1.
Поостановная трасса для построения матрицы:
               stop_id                       stop_name
stop_sequence                                         
1                  308  пос. Михалково (ул. Колхозная)
2                  331                       Колхозная
3                  345           Почта (пос. Лихвинка)
4                  365         Переезд (пос. Лихвинка)
5                  379                          Склады
6                  387                        Развилка
7                  419                            Депо
8                  445         пл. Московского вокзала
Загружаем данные о числе рейсов по указанным вариантам поостановочной трассы...


C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


...нет данных о числе рейсов....
...считаем, что по каждому варианту был совершен один рейс.
Загружаем исходные данные о посадке-высадке:
               stop_id  pass_in  pass_out
stop_sequence                            
1                  308      1.3       0.0
2                  331      2.0       0.0
3                  345      0.9       0.3
4                  365      2.9       0.0
5                  379      0.2       0.4
6                  387      0.2       1.1
7                  419      1.3       1.6
8                  445      0.0       5.4
Суммарно за рейс:
pass_in     8.9
pass_out    8.9
Вносим поправки в исходные данные для корректного вычисления матрицы...
...наименьший порядковый номер о.п. в поостановочной трассе -- 1...
...наибольший порядковый номер о.п. в поостановочной трассе -- 8...
...зафиксировано -0.0 пассажиров в салоне тс, увеличиваем посадку на начальном о.п...
...корректрировка выполнена:
               stop_id  pass_in  pass_out
stop_sequence              

(1,
    stop_sequence  stop_id                       stop_name   pass_in  pass_out
 0              1      308  пос. Михалково (ул. Колхозная)  1.333333  0.000000
 1              2      331                       Колхозная  2.000000  0.000000
 2              3      345           Почта (пос. Лихвинка)  0.888889  0.333333
 3              4      365         Переезд (пос. Лихвинка)  2.888889  0.000000
 4              5      379                          Склады  0.222222  0.444444
 5              6      387                        Развилка  0.222222  1.111111
 6              7      419                            Депо  1.333333  1.555556
 7              8      445         пл. Московского вокзала  0.000000  5.444444,
     start_num  end_num  start_id                      start_name  end_id  \
 0           1        1       308  пос. Михалково (ул. Колхозная)     308   
 1           1        2       308  пос. Михалково (ул. Колхозная)     331   
 2           1        3       308  пос. Михалково (ул

# Part C2

In [30]:
# задаем директорию с исходными данными
args['source'] = 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/'
path = args['source']


# !!!!!!
# задаем директорию с результатами
args['results'] = ('C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/преобразованные '
+ timing +'/' + type_of_day +'/')


# логин и пароль для подключения к базе UARMS
args['username'] = None
args['password'] = None
args['period-type'] = None
args

{'all_sources': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/mvns_files/',
 'alpha': 2,
 'beta': 1000,
 'log': None,
 'one-stop-dist': 300,
 'password': None,
 'period-type': None,
 'results': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/преобразованные Утро пик/буд/',
 'source': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/обработанные/',
 'summer': '05.01-09.30',
 'time': ['00:00',
  '01:00',
  '02:00',
  '03:00',
  '04:00',
  '05:00',
  '06:00',
  '07:00',
  '08:00',
  '09:00',
  '10:00',
  '11:00',
  '12:00',
  '13:00',
  '14:00',
  '15:00',
  '16:00',
  '17:00',
  '18:00',
  '19:00',
  '20:00',
  '21:00',
  '22:00',
  '23:00'],
 'username': None,
 'winter': '10.01-04.30',
 'workdays': 'C:/Users/popova_kv/Desktop/New/scripts/Tula_matrix_from_detour/workdays.csv'}

In [31]:
# запускаем расчет для всех файлов в директории включая вложенные директории

# запустил для всех мвн 
# (из апи мосгортранса обработаны архивы с 20161220 по 20180228)
# (по коммерческим перевозчикам обработаны данные за 4 кв 2017 года)
# (дополнительно подготовлдены данные по некоторым маршрутам мосгортранса, которых нет в апи)
# время выполнения составляет ЧЧ:ММ:СС (1977 мвн'ов)
# - 1 мвн обработано с ошибкой (0.1%)
# - 486 мвн'у не было совершено ни одного рейса (24.5%)
# - и только 1054 мвн'ов имеют 5 и более рейсов (53.3%)

ts_start = pd.datetime.now()
lst_mvns = list()
lst_nraces = list()
df_matrix = None
df_inout = None
for file in tqdm(os.listdir(path=path)):
    
    
    # !!!!!!
    
#      & (file.split('.')[0].split('-')[-2] == type_of_day)
    if (file.split('.')[-1] == 'xlsx') & (file.split('.')[0].split('-')[-1] == timing):
        mvn = file[:-5]
        lst_mvns.append(mvn)
        args['source'] = path + '/' + file
        args['log'] = mvn + '.log'
        try:
            # (число рейсов, исх-посадка-высадка, матрица)
            (n, inout, matrix) = main_args_C1(args)
            # добавляем данные об исходной посадке-высадке
            if inout is not None:
                inout['mvn'] = mvn
            if df_inout is None:
                    df_inout = inout
            else:
                df_inout = pd.concat([df_inout, inout], ignore_index=True)
            # добавляем данные по поездкам между о.п.
            if matrix is not None:
                matrix['mvn'] = mvn
            if df_matrix is None:
                df_matrix = matrix
            else:
                df_matrix = pd.concat([df_matrix, matrix], ignore_index=True)
            lst_nraces.append(n)
        except:
            lst_nraces.append('ERROR')

C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is dep

C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is dep

C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is dep

C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is dep

C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\popova_kv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is dep

In [32]:
file

'Тм_9-Прямое-буд-Утро пик.xlsx'

In [33]:
df_results = pd.DataFrame({'mvn': lst_mvns, 'nraces': lst_nraces})
df_results.to_csv(args['results'] + '/log' + ts_start.strftime('-%Y-%m-%d-%H-%M')+ '.csv', sep=';', index=False)

In [34]:
df_matrix\
    .rename(columns={'start_num':'departure_stop_seq', 'start_id':'departure_stop_id',\
        'end_num':'arrival_stop_seq', 'end_id':'arrival_stop_id', 'value' : 'pass'})\
    [['mvn','departure_stop_seq', 'departure_stop_id', 'arrival_stop_seq', 'arrival_stop_id', 'pass']]\
    .dropna(subset=['pass'])\
    .sort_values(['mvn','departure_stop_seq','arrival_stop_seq'])\
    .to_csv('%s/matrix.csv' % args['results'], sep=';', encoding='cp1251', float_format='%.3f', index=False)

In [35]:
df_inout\
    [['stop_id','mvn','stop_sequence','pass_in','pass_out']]\
    .sort_values(['stop_id','mvn','stop_sequence'])\
    .to_csv('%s/inout.csv' % args['results'], sep=';', encoding='cp1251', float_format='%.3f', index=False)

In [36]:
df_matrix.groupby(['start_id','end_id'])[['value']].sum().to_csv('%s/matrix_summed.csv' % args['results'], sep=';', float_format='%.3f')

#### Проставление частот

In [52]:
matrix_work_morn = pd.read_csv('matrix_work_morn.csv', encoding='cp1251', sep=';')
matrix_work_morn

,mvn,departure_stop_seq,departure_stop_id,arrival_stop_seq,arrival_stop_id,pass
0,1-Обратное-буд-Утро пик,1,785,2,757,0.000
1,1-Обратное-буд-Утро пик,1,785,3,738,0.000
2,1-Обратное-буд-Утро пик,1,785,4,708,0.000
3,1-Обратное-буд-Утро пик,1,785,5,710,0.000
4,1-Обратное-буд-Утро пик,1,785,6,700,0.000
5,1-Обратное-буд-Утро пик,1,785,7,714,0.000
6,1-Обратное-буд-Утро пик,1,785,8,720,0.000
7,1-Обратное-буд-Утро пик,1,785,9,711,0.015
8,1-Обратное-буд-Утро пик,1,785,10,717,0.006
9,1-Обратное-буд-Утро пик,1,785,11,669,0.010


In [53]:
matrix_work_morn['pass'].sum()

6880.075000000001

In [54]:
numoftrips = pd.read_csv('numoftrips.csv', encoding='cp1251', sep=';')
numoftrips

,route_name,Type,mvn_old,mvn,num_of_trips
0,К_102К,A,К-102К-A-Прямое,К_102К-Прямое-буд-Утро пик,2
1,К_102К,A,К-102К-A-Обратное,К_102К-Обратное-буд-Утро пик,2
2,1,A,1-A-Прямое,1-Прямое-буд-Утро пик,3
3,2,A,2-A-Прямое,2-Прямое-буд-Утро пик,2
4,3,A,3-A-Прямое,3-Круговой-буд-Утро пик,5
5,5,A,5-A-Прямое,5-Прямое-буд-Утро пик,2
6,11,A,11-A-Прямое,11-Прямое-буд-Утро пик,4
7,12,A,12-A-Прямое,12-Прямое-буд-Утро пик,3
8,13,A,13-A-Прямое,13-Прямое-буд-Утро пик,2
9,16,A,16-A-Прямое,16-Прямое-буд-Утро пик,3


In [55]:
matrix_work_morn = matrix_work_morn.merge(numoftrips[['mvn','num_of_trips']], how = 'left', on=['mvn'])
matrix_work_morn

,mvn,departure_stop_seq,departure_stop_id,arrival_stop_seq,arrival_stop_id,pass,num_of_trips
0,1-Обратное-буд-Утро пик,1,785,2,757,0.000,3
1,1-Обратное-буд-Утро пик,1,785,3,738,0.000,3
2,1-Обратное-буд-Утро пик,1,785,4,708,0.000,3
3,1-Обратное-буд-Утро пик,1,785,5,710,0.000,3
4,1-Обратное-буд-Утро пик,1,785,6,700,0.000,3
5,1-Обратное-буд-Утро пик,1,785,7,714,0.000,3
6,1-Обратное-буд-Утро пик,1,785,8,720,0.000,3
7,1-Обратное-буд-Утро пик,1,785,9,711,0.015,3
8,1-Обратное-буд-Утро пик,1,785,10,717,0.006,3
9,1-Обратное-буд-Утро пик,1,785,11,669,0.010,3


In [49]:
from datetime import datetime
matrix_work_morn.to_csv('matrix_work_morn-{:%Y-%m-%d-(%H-%M)}.csv'.format(datetime.now()), sep=';', encoding='cp1251', index=False)

In [56]:
matrix_work_morn['pass_freq'] = matrix_work_morn['pass'].astype(float) * matrix_work_morn['num_of_trips'].astype(float)
matrix_work_morn

,mvn,departure_stop_seq,departure_stop_id,arrival_stop_seq,arrival_stop_id,pass,num_of_trips,pass_freq
0,1-Обратное-буд-Утро пик,1,785,2,757,0.000,3,0.000
1,1-Обратное-буд-Утро пик,1,785,3,738,0.000,3,0.000
2,1-Обратное-буд-Утро пик,1,785,4,708,0.000,3,0.000
3,1-Обратное-буд-Утро пик,1,785,5,710,0.000,3,0.000
4,1-Обратное-буд-Утро пик,1,785,6,700,0.000,3,0.000
5,1-Обратное-буд-Утро пик,1,785,7,714,0.000,3,0.000
6,1-Обратное-буд-Утро пик,1,785,8,720,0.000,3,0.000
7,1-Обратное-буд-Утро пик,1,785,9,711,0.015,3,0.045
8,1-Обратное-буд-Утро пик,1,785,10,717,0.006,3,0.018
9,1-Обратное-буд-Утро пик,1,785,11,669,0.010,3,0.030


In [57]:
matrix_work_morn['pass_freq'].sum()

33875.725

In [63]:
sum_matrix = matrix_work_morn[['departure_stop_id', 'arrival_stop_id', 'pass_freq']].groupby(['departure_stop_id', 'arrival_stop_id'])['pass_freq'].agg('sum').reset_index()
sum_matrix = sum_matrix.sort_values(by = ['pass_freq'], ascending = False).reset_index(drop=True)
sum_matrix

,departure_stop_id,arrival_stop_id,pass_freq
0,660,115,110.568
1,652,115,61.123
2,660,103,50.113
3,467,114,46.160
4,660,614,43.773
5,488,114,41.647
6,15,600,40.353
7,741,101,36.274
8,514,447,35.575
9,515,114,35.540


In [60]:
sum_matrix['pass_freq'].sum()

33875.72499999999

In [80]:
from_matrix = sum_matrix.groupby(['departure_stop_id'])['pass_freq'].agg('sum').reset_index()
to_matrix = sum_matrix.groupby(['arrival_stop_id'])['pass_freq'].agg('sum').reset_index()


from_matrix = from_matrix.sort_values(by = ['pass_freq'], ascending = False).reset_index(drop=True)
to_matrix = to_matrix.sort_values(by = ['pass_freq'], ascending = False).reset_index(drop=True)
new_matrix = from_matrix.merge(to_matrix, how = 'outer', left_on = 'departure_stop_id', right_on = 'arrival_stop_id')
new_matrix['k_dep_arr'] = 1 / (new_matrix['pass_freq_x'] / new_matrix['pass_freq_y'])
new_matrix = new_matrix.replace([np.inf, -np.inf], np.nan)
# new_matrix = new_matrix.sort_values(by = ['k_dep_arr'], ascending = True).reset_index(drop=True)
new_matrix

,departure_stop_id,pass_freq_x,arrival_stop_id,pass_freq_y,k_dep_arr
0,660.0,633.147,660.0,245.333,0.387482
1,9.0,586.973,9.0,121.604,0.207171
2,15.0,417.947,15.0,196.970,0.471280
3,652.0,415.780,652.0,185.761,0.446777
4,741.0,397.857,741.0,187.193,0.470503
5,467.0,396.901,467.0,105.575,0.265998
6,103.0,331.522,103.0,254.598,0.767967
7,11.0,326.428,11.0,156.170,0.478421
8,445.0,318.123,445.0,162.928,0.512154
9,17.0,310.376,17.0,103.748,0.334266
